In [2]:
import numpy
from numpy import linalg

In [3]:
def smartReverse(a, na, x, i):
    n = x.size
    
    ha = a.copy()
    ha[:, i] = x
    
    l = na * x
    if l[i, 0] == 0:
        print('Матрица необратима')
    
    wl = l.copy()
    wl[i, 0] = -1
    
    dl = (-1 / l[i])[0,0] * wl
    q = numpy.mat(numpy.eye(n))
    q[:, i] = dl[:,0]
    return q * na

In [100]:
infynity = 10000000

def simplex(a, c, x, ib, du, dd):
    n = len(x)
    inb = set(range(n)) - set(ib)
    ab = a[:, ib]
    nab = linalg.inv(ab)
    m = len(ib)
    
    
    while True:
        print("ITER")
        cb = c[ib, :]
        ut = cb.transpose() * nab
        d = ut * a - c.transpose()

        for i in inb:
            if (d[0, i] < 0  and x[i]<dd[i]+0.001) or (d[0, i] > 0  and x[i]>du[i]-0.001):
            #if (d[0, i] < 0  and x[i]==dd[i]) or (d[0, i] > 0  and x[i]==du[i]):
                j0 = i
                print(j0)
                break
        else:
            return x, ib, nab

        z0 = 1 if d[0, j0] < 0 else -1
        print('z0:',z0)
        z = -z0 * nab * a[:, j0]
        print('z:',z)
        zz = nab * a[:, j0]
        o = []
        for i in range(m):
            if z[i, 0] > 0:
                o.append((du[ib[i]] - x[ib[i]]) / z[i, 0])
            elif z[i, 0] < 0:
                o.append((dd[ib[i]] - x[ib[i]]) / z[i, 0])
            else:
                o.append(infynity)
        
        print('o:',o)
        s = -1
        o0 = infynity
        for i in range(len(ib)):
            if o[i] < o0:
                s = i
                o0 = o[i]
        if s == -1:
            return None, None, None
        
        print('o0:',o0)

        #x[j0] = o0
        for i in range(len(ib)):
            x[ib[i]] = x[ib[i]] + o0 * z[i,0]
            
        x[j0] = x[j0] + o0 * z0
        ib[s] = j0
        #for i in set(range(n)) - set(ib):
        #    x[i] = 0
        

        nab = smartReverse(ab, nab, a[:,j0], s)

        #zzs = zz[s]
        #zz[s] = -1
        #ds = -zz / zzs
        #M = numpy.matrix(numpy.eye(m), dtype=numpy.float64)
        #M[:, s] = ds
        #nab = M * nab
        #print()
        
        ab[:,s] = a[:,j0]
        print(x)
        

In [43]:
a = numpy.mat( [[1, -1, 0],[0,1,0], [0,0,1]] , dtype=numpy.float64)
na = numpy.mat( [[1, 1, 0],[0,1,0], [0,0,1]] , dtype=numpy.float64)
x = numpy.mat([[1], [0], [1]])
smartReverse(a, na, x, 2)

matrix([[ 1.,  1., -1.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  1.]])

In [44]:
a = numpy.mat( [[-1, 1, 1, 0, 0],[1, 0, 0, 1, 0], [0, 1, 0, 0, 1]] , dtype=numpy.float64)
c = numpy.mat( [[1], [1], [0], [0], [0]], dtype=numpy.float64 )
x = [0, 0, 1, 3, 2]
ib = [2, 3, 4]
simplex(a, c, x, ib)

TypeError: simplex() missing 2 required positional arguments: 'du' and 'dd'

In [8]:
a = numpy.mat( [[-2, -1, 1, -7, 0, 0, 0, 2],
                [4, 2, -1, 0, 1, 5, -1, -5],
                [1, 11, 0, 1, 0, 3, 1, 1]] , 
              dtype=numpy.float64)
c = numpy.mat( [[6], [-9], [5], [-2], [6], [0], [-1], [3]], dtype=numpy.float64 )
x = [4, 0, 6, 0, 4, 0, 0, 0]
ib = [0, 2, 4]
v, ind = simplex(a, c, x, ib)
print('Best x: ',v)
print('Max fun: ',(v*c)[0,0])

ValueError: too many values to unpack (expected 2)

In [9]:
def simplex_start(a, b, c):
    n = len(c)
    m = len(b)
    for i in range(m):
        if b[i] < 0:
            b[i] *= -1
            a[i] = -a[i]
    
    
    dop_c = numpy.mat([0] * n + [-1] * m).transpose()
    dop_a = numpy.concatenate((a, numpy.eye(m)), axis=1)
    x = [0] * n + b
    ib = list(range(n, n+m))
    
    while True:
        
        v, ib, nab = simplex(dop_a, dop_c, x, ib)
        
        if (numpy.array(v[n:n+m]) != 0).any():
            print('Задача несовместна')
        
        if not (set(ib) and set(range(n, n+m))):
            break
            
        for k in range(len(ib)):
            if ib[k] >= n:
                j = ib[k] - n
                for i in set(range(n)) - set(ib):
                    l = nab * a[:, i]
                    if l[k] != 0:
                        ib[k] = j
                        break
                else:
                    idx = [True] * m
                    idx[i] = False
                    a = a[idx, :]
                    dop_a = dop_a[idx, :]
                    del b[i]
                    n = n - 1
                    x = [0] * n + b
                    ib = range(n, n+m)
                    break
        else:
            return simplex(a, c, x[:n], ib)

In [10]:
a = numpy.mat( [[-2, -1, 1, -7, 0, 0, 0, 2],
                [4, 2, -1, 0, 1, 5, -1, -5],
                [1, 11, 0, 1, 0, 3, 1, 1]] , 
              dtype=numpy.float64)
c = numpy.mat( [[6], [-9], [5], [-2], [6], [0], [-1], [3]], dtype=numpy.float64 )
x = [4, 0, 6, 0, 4, 0, 0, 0]
ib = [0, 2, 4]
b = [-2, 14, 4]


v, ind, _ = simplex_start(a, b, c)
print('Best x: ',v)
print('Max fun: ',(v*c)[0,0])

Best x:  [0, 0, 26.0, 4.0, 40.0, 0, 0, 0]
Max fun:  362.0


In [11]:
a = numpy.mat( [[2, -1, 1, -7.5, 0, 0, 0, 2],
                [4, 2, -1, 0, 1, 5, -1, -4],
                [1, -1, 1, -1, 0, 3, 1, 1]] , 
              dtype=numpy.float64)
c = numpy.mat( [[-6], [-9], [-5], [2], [-6], [0], [1], [3]], dtype=numpy.float64 )
x = [4, 0, 6, 0, 4, 0, 0, 0]
ib = [0, 2, 4]
b = [14, 14, 10]


v, ind, _ = simplex_start(a, b, c)
print('Best x: ',v)
print('Max fun: ',(v*c)[0,0])

Best x:  [4.6249999999999991, 0, 0, 0, 0, 1.0000000000000002, 0, 2.375]
Max fun:  -20.625


In [12]:
a = numpy.mat( [[0, 1, 4, 1, 0, -8, 1, 5],
                [0, -1, 0, -1, 0, 0, 0, 0],
                [0, 2, -1, 0, -1, 3, -1, 0],
                [1, 1, 1, 1, 0, 3, 1, 1]
               ] , 
              dtype=numpy.float64)
c = numpy.mat( [[-5], [2], [3], [-4], [-6], [0], [1], [5]], dtype=numpy.float64 )
b = [36, -11, 10, 20]


v, ind, _ = simplex_start(a, b, c)
print('Best x: ',v)
print('Max fun: ',(v*c)[0,0])

Best x:  [0, 9.4999999999999982, 5.3333333333333339, 1.5000000000000009, 0, 0, 3.6666666666666647, 0]
Max fun:  32.6666666667


In [13]:
c = numpy.mat([[-4], [-3], [-7], [0], [0]], dtype=numpy.float64)
a = numpy.mat([[-2, -1, -4, 1, 0],
              [-2, -2, -2, 0, 1]], dtype=numpy.float64)
b = [-1, -3/2]

v, ind, _ = simplex_start(a, b, c)
print('Best x: ',v)
print('Max fun: ',(v*c)[0,0])

Best x:  [0.25, 0.5, 0, 0, 0]
Max fun:  -2.5


In [56]:
c = numpy.mat( [[2], [2], [1], [-10], [1], [4], [-2], [-3]])

a = numpy.mat([
              [-2, -1, 1, -7, 0, 0, 0, 2],
              [4, 2, 1, 0, 1, 5, -1, -5],
              [1, 1, 0, -1, 0, 3, -1, 1]
              ]
              ,dtype=numpy.float64
             )

b = [-2, 4, 3]

v, ind, _ = simplex_start(a, b, c)
print('Best x: ',v)
print('Max fun: ',(v*c)[0,0])

NameError: name 'simplex_start' is not defined

In [103]:
c = numpy.mat( [[1], [4], [4], [1], [1], [5], [1]])

a = numpy.mat([
              [1, 2, 0, -1, 0, -2, 1],
              [0, -1, 1, -1, 0, 3, 2],
              [0, 4, 0, -1, 1, 6, -4]
              ]
              ,dtype=numpy.float64
             )

x = [1, 0, 2, 4, 3, 5, 0]
ib = [0, 2, 4]

dd = [0, 0, 0, 0, 0, 0, 0]
du = [5, 6, 4, 4, 5, 5, 6]

x, ib, _ = simplex(a, c, x, ib, du, dd)
print('Best x: ',x)
print('Max fun: ',(x*c)[0,0])

x, ib, _ = simplex(a, c, x, ib, du, dd)
print('Best x: ',x)
print('Max fun: ',(x*c)[0,0])

print(a*numpy.matrix(x).T)

ITER
1
z0: 1
z: [[-2.]
 [ 1.]
 [-4.]]
o: [0.5, 2.0, 0.75]
o0: 0.5
[0.0, 0.5, 2.5, 4, 1.0, 5, 0]
ITER
5
z0: -1
z: [[ -1.]
 [  2.]
 [ 10.]]
o: [0.5, 0.75, 0.40000000000000002]
o0: 0.4
[0.0, 0.099999999999999978, 3.2999999999999998, 4, 5.0, 4.5999999999999996, 0]
ITER
Best x:  [0.0, 0.099999999999999978, 3.2999999999999998, 4, 5.0, 4.5999999999999996, 0]
Max fun:  45.6
ITER
Best x:  [0.0, 0.099999999999999978, 3.2999999999999998, 4, 5.0, 4.5999999999999996, 0]
Max fun:  45.6
[[-13.]
 [ 13.]
 [ 29.]]
